In [1]:
import pandas as pd

In [2]:
path = r"D:/DataLovesAcademy/Final_project/medical_insurance.csv"
df = pd.read_csv(path)

In [3]:
df.head(5)

,person_id,age,sex,region,urban_rural,income,education,marital_status,employment_status,household_size,...,liver_disease,arthritis,mental_health,proc_imaging_count,proc_surgery_count,proc_physio_count,proc_consult_count,proc_lab_count,is_high_risk,had_major_procedure
0,75722,52,Female,North,Suburban,22700.0,Doctorate,Married,Retired,3,...,0,1,0,1,0,2,0,1,0,0
1,80185,79,Female,North,Urban,12800.0,No HS,Married,Employed,3,...,0,1,1,0,0,1,0,1,1,0
2,19865,68,Male,North,Rural,40700.0,HS,Married,Retired,5,...,0,0,1,1,0,2,1,0,1,0
3,76700,15,Male,North,Suburban,15600.0,Some College,Married,Self-employed,5,...,0,0,0,1,0,0,1,0,0,0
4,92992,53,Male,Central,Suburban,89600.0,Doctorate,Married,Self-employed,2,...,0,1,0,2,0,1,1,0,1,0


Аналіз клієнтів, що належать до групи із найвищими витратами на страховку, портрет такого клієнта

In [4]:
import numpy as np

In [5]:
# прибираю всіх, кому меншее 18
df = df[df['age'] >= 18].copy() 

In [15]:
# закодовую курців та стать
smoker_mapping = {'Never': 0, 'Former': 1, 'Current': 2}
df['smoker_cod'] = df['smoker'].map(smoker_mapping)

sex_mapping = {'Male': 0, 'Female': 1, 'Other': 2} 
df['sex_cod'] = df['sex'].map(sex_mapping)

In [16]:
percentile_90 = df['annual_medical_cost'].quantile(0.90)
df['High_Cost_Client'] = np.where(df['annual_medical_cost'] >= percentile_90, 1, 0)

print(f"90-й перцентиль витрат (поріг High Cost): ${percentile_90:,.2f}")

90-й перцентиль витрат (поріг High Cost): $6,268.56


In [17]:
profile_analysis = df.groupby('High_Cost_Client').agg(
    count=('person_id', 'count'),
    avg_age=('age', 'mean'),
    avg_bmi=('bmi', 'mean'),
    pct_smokers_current=('smoker', lambda x: (x == 'Current').mean() * 100), # Частка поточних курців у групі
    avg_cost=('annual_medical_cost', 'mean')
)

print("\nПрофіль клієнта High Cost (0 = Low, 1 = High):")
print(profile_analysis.round(2))


Профіль клієнта High Cost (0 = Low, 1 = High):
                  count  avg_age  avg_bmi  pct_smokers_current  avg_cost
High_Cost_Client                                                        
0                 87260    48.23    26.95                10.81   2238.46
1                  9696    52.44    27.38                23.53  10167.42


Аналіз факторів, які найсильніше впливають на вартість страхування

In [19]:
correlation_features = [
    'annual_medical_cost',
    'age',
    'bmi',
    'dependents',
    'smoker_encoded',
    'sex_encoded',
    'hypertension', 
    'diabetes',
    'copd',
    'mental_health'
]

# Розрахунок кореляційної матриці з цільовою змінною 'annual_medical_cost'
correlation_matrix = df.loc[:, correlation_features].corr()

# Вибір лише рядка з кореляцією до 'annual_medical_cost' та сортування
cost_correlation = correlation_matrix['annual_medical_cost'].sort_values(ascending=False).drop('annual_medical_cost')

print("\nКореляція ознак з 'annual_medical_cost':")
print(cost_correlation.round(3))


Кореляція ознак з 'annual_medical_cost':
smoker_encoded    0.156
hypertension      0.154
age               0.127
mental_health     0.126
diabetes          0.107
copd              0.070
bmi               0.043
dependents        0.004
sex_encoded      -0.004
Name: annual_medical_cost, dtype: float64


In [20]:
df.to_csv('final_file.csv', index=False)
print("Файл 'final_file.csv' успішно збережено.")

Файл 'final_file.csv' успішно збережено.
